In [2]:
# ---------------------------
# Cell 1 — install dependencies
# ---------------------------
!pip install pytrends pandas openpyxl requests xlrd


In [3]:
# ---------------------------
# Cell 2 — imports & utils
# ---------------------------
import time
import requests
import pandas as pd
from pytrends.request import TrendReq

# If running in Google Colab and you want to provide a download link
try:
    from google.colab import files as colab_files
    _IN_COLAB = True
except Exception:
    _IN_COLAB = False

# basic helper to safely call request endpoints
def safe_get(url, params=None, headers=None, retries=3, backoff=1.0):
    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, headers=headers, timeout=15)
            r.raise_for_status()
            return r
        except Exception as e:
            if attempt + 1 == retries:
                raise
            time.sleep(backoff * (attempt + 1))


In [4]:
# ---------------------------
# Cell 3 — source functions
# ---------------------------
# Google Trends (pytrends)
pytrends = TrendReq(hl='ru-RU', tz=180)

def get_google_trends_related(keyword, geo='RU', timeframe='today 12-m'):
    results = []
    try:
        pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo=geo)
        related = pytrends.related_queries().get(keyword)
        if related:
            top = related.get('top')
            # top can be None if no top queries
            if top is not None:
                for _, row in top.iterrows():
                    results.append({
                        "Ключевая фраза": row["query"],
                        "Источник": "Google Trends",
                        "Популярность (справочно)": row.get("value", None)
                    })
    except Exception as e:
        print(f"[Google Trends] error for '{keyword}': {e}")
    return results

# Google Suggest (autocomplete)
def get_google_suggest(keyword, hl="ru"):
    url = "http://suggestqueries.google.com/complete/search"
    params = {
        "client": "firefox",
        "hl": hl,
        "q": keyword
    }
    headers = {
        "User-Agent": "keyword-agent/1.0"
    }
    try:
        r = safe_get(url, params=params, headers=headers)
        suggestions = r.json()[1]
        return [{"Ключевая фраза": s, "Источник": "Google Suggest", "Популярность (справочно)": None}
                for s in suggestions]
    except Exception as e:
        print(f"[Google Suggest] error for '{keyword}': {e}")
        return []

# Yandex Suggest
def get_yandex_suggest(keyword, lang="ru"):
    url = "https://suggest.yandex.net/suggest-ya.cgi"
    params = {
        "part": keyword,
        "lang": lang,
        "uil": lang,
        "v": "4",
        "search_type": "suggest"
    }
    headers = {
        "User-Agent": "keyword-agent/1.0"
    }
    try:
        r = safe_get(url, params=params, headers=headers)
        # Yandex returns JSON array; suggestions usually at index 1
        suggestions = r.json()[1]
        return [{"Ключевая фраза": s, "Источник": "Yandex Suggest", "Популярность (справочно)": None}
                for s in suggestions]
    except Exception as e:
        print(f"[Yandex Suggest] error for '{keyword}': {e}")
        return []


In [5]:
# ---------------------------
# Cell 4 — simple theme / intent classifier
# ---------------------------
# This is a rule-based quick classifier to fill "Тема" and "ЦА"
STYLE_KEYWORDS = {
    "реализм": ["реализм", "реалист", "реалистич"],
    "абстракция": ["абстракц", "абстракт"],
    "пейзаж": ["пейзаж", "ландшафт"],
    "портрет": ["портрет"],
    "масло": ["маслом", "маслян"],
    "графика": ["график", "графика"],
    "современное": ["современн", "современная"]
}

INTENT_KEYWORDS = {
    "купить": ["купить", "куплю", "продажа", "заказ"],
    "заказать": ["заказать", "сделать на заказ"],
    "в интерьере": ["интерьер", "для интерьера", "в интерьере"],
    "в подарок": ["подарок", "в подарок", "подарить"],
    "выбрать": ["выбрать", "подобрать"]
}

def classify_phrase(phrase):
    phrase_l = phrase.lower()
    topics = []
    intents = []
    for t, tokens in STYLE_KEYWORDS.items():
        for tk in tokens:
            if tk in phrase_l:
                topics.append(t)
                break
    for i, tokens in INTENT_KEYWORDS.items():
        for tk in tokens:
            if tk in phrase_l:
                intents.append(i)
                break
    topic = ", ".join(topics) if topics else ""
    intent = ", ".join(intents) if intents else ""
    # target audience guess: simple heuristic — if contains 'купить' or 'в подарок' -> buyers
    audience = ""
    if "купить" in phrase_l or "продажа" in phrase_l or "куплю" in phrase_l:
        audience = "Покупатели"
    elif "для интерьера" in phrase_l or "интерьер" in phrase_l:
        audience = "Декораторы / владельцы жилья"
    elif "подарок" in phrase_l:
        audience = "Покупатели подарков"
    return topic, intent, audience


In [6]:
# ---------------------------
# Cell 5 — main agent orchestration
# ---------------------------
def keyword_agent(seeds,
                  output_file="keywords_output.xlsx",
                  use_trends=True,
                  use_google_suggest=True,
                  use_yandex_suggest=True,
                  sleep_between=0.8):
    collected = []
    seen = set()

    for seed in seeds:
        seed = seed.strip()
        if not seed:
            continue

        # Google Trends
        if use_trends:
            trends = get_google_trends_related(seed)
            for rec in trends:
                phrase = rec["Ключевая фраза"].strip()
                key = (phrase, rec["Источник"])
                if key not in seen:
                    seen.add(key)
                    topic, intent, audience = classify_phrase(phrase)
                    rec.update({"Тема": topic, "Целевая аудитория": audience, "Intent_tags": intent})
                    collected.append(rec)
            time.sleep(sleep_between)

        # Google Suggest
        if use_google_suggest:
            gs = get_google_suggest(seed)
            for rec in gs:
                phrase = rec["Ключевая фраза"].strip()
                key = (phrase, rec["Источник"])
                if key not in seen:
                    seen.add(key)
                    topic, intent, audience = classify_phrase(phrase)
                    rec.update({"Тема": topic, "Целевая аудитория": audience, "Intent_tags": intent})
                    collected.append(rec)
            time.sleep(sleep_between)

        # Yandex Suggest
        if use_yandex_suggest:
            ys = get_yandex_suggest(seed)
            for rec in ys:
                phrase = rec["Ключевая фраза"].strip()
                key = (phrase, rec["Источник"])
                if key not in seen:
                    seen.add(key)
                    topic, intent, audience = classify_phrase(phrase)
                    rec.update({"Тема": topic, "Целевая аудитория": audience, "Intent_tags": intent})
                    collected.append(rec)
            time.sleep(sleep_between)

    # Build DataFrame and save
    df = pd.DataFrame(collected)
    # Reorder columns
    cols = ["Ключевая фраза", "Источник", "Тема", "Intent_tags", "Целевая аудитория", "Популярность (справочно)"]
    for c in cols:
        if c not in df.columns:
            df[c] = ""
    df = df[cols]
    df.to_excel(output_file, index=False)
    print(f"Saved {len(df)} rows to {output_file}")
    return df


In [7]:
# ---------------------------
# Cell 6 — run the agent (edit seeds as needed)
# ---------------------------
seeds = [
    "картины",
    "живопись",
    "графика",
    "покупка картин",
    "искусство",
    "художник"
]

df = keyword_agent(seeds, output_file="keywords_output.xlsx")
df.head(30)


Saved 237 rows to keywords_output.xlsx


,Ключевая фраза,Источник,Тема,Intent_tags,Целевая аудитория,Популярность (справочно)
0,описание картины,Google Trends,,,,100.0
1,картины художников,Google Trends,,,,71.0
2,картины по номерам,Google Trends,,,,64.0
3,известные картины,Google Trends,,,,38.0
4,ван гог картины,Google Trends,,,,22.0
5,картины пикассо,Google Trends,,,,22.0
6,картины дали,Google Trends,,,,22.0
7,дали картины,Google Trends,,,,22.0
8,картины моне,Google Trends,,,,18.0
9,васнецов картины,Google Trends,,,,17.0


In [8]:
# ---------------------------
# Cell 7 — download the file (Colab)
# ---------------------------
# If running in Colab, this will provide a download link
if _IN_COLAB:
    colab_files.download("keywords_output.xlsx")
else:
    print("Not in Colab: file 'keywords_output.xlsx' saved to current directory.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>